# 环境配置
--------------------------------------------------------
```python
pip -m pip install --upgrade pip
# 更换 pypi 源加速库的安装
pip config set global.index-url https://pypi.tuna.tsinghua.edu.cn/simple

pip install torch==2.5.1
pip install torchvision==0.20.1
pip install swanlab==0.3.23
pip install scikit-learn==1.5.2
pip install pandas==2.0.3
pip install matplotlib==3.7.2
```
--------------------------------------------------------


或者你也可以使用 `conda` 来管理你的环境
------------------------------------------------------------
``` python
conda create -n lstm python==3.10

conda activate lstm

pip install uv && uv pip install -r requirements.txt
```
------------------------------------------------------------

包的引入

In [ ]:
import numpy as np
import torch

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

超参数的定义，这里显式的给出，在之后的代码里我们将使用 swanlab 来管理超参数。

In [3]:
# Hyperparameters
vocab_size = 30
learning_rate = 0.005
hidden_units = 128

一些网络中会用到的激活函数，这里给出主要是为了展示其定义和数学表达，实际中不论是 numpy 库，还是 pytorch 库，都已经帮我们实现好了，我们直接调用即可。

In [4]:
# Activation Functions
#sigmoid function
def sigmoid(X):
    return 1/1(1+np.exp(-X))

def tanh_activation(X):
    return np.tanh(X)

# softmax activation
def softmax(X):
    exp_X = np.exp(X)
    exp_X_sum = np.sum(exp_X, axis=1).reshape(-1, 1)
    exp_X = exp_X / exp_X_sum
    return exp_X

<>:4: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:4: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
/var/folders/xy/2nl06h6134z8d63822qpjs840000gn/T/ipykernel_48493/1941595322.py:4: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  return 1/1(1+np.exp(-X))


![](../assets/rnn_vs_lstm.png)
这一部分进行LSTM 网络状态的初始化，由上图以及前面学习的知识可知，LSTM 网络的状态有两个，一个是 $C_t$，一个是 $H_t$，我们需要分别对这两个状态进行初始化，并封装成一个函数`init_lstm_state`

In [5]:
# 初始化 lstm，包含cell state, hidden state
def init_lstm_state(batch_size, hidden_units, device):
    return (torch.zeros((batch_size, hidden_units), device=device), 
            torch.zeros((batch_size, hidden_units), device=device))

下面一部分进行各参数的初始化，首先定义一个`normal`函数用于生成满足正态分布的Tensor形式的数据。

后面再定义包含遗忘门，输入门，输出门，候选记忆单元，隐藏层/输出层的参数。并使用一个参数字典统一管理。

In [6]:
# initialize parameters
def initialize_parameters(vocab_size, hidden_units, device):
    std = 0.01
    input_units = output_units = vocab_size

    # 正态分布
    def normal(shape):
        return torch.randn(size=shape, device=device) * std

    # LSTM cell weights
    forget_gate_weights = normal((input_units + hidden_units, hidden_units))
    input_gate_weights = normal((input_units + hidden_units, hidden_units))
    output_gate_weights = normal((input_units + hidden_units, hidden_units))
    c_tilda_gate_weights = normal((input_units + hidden_units, hidden_units))

    # 偏置项
    forget_gate_bias = torch.zeros((1, hidden_units), device=device)
    input_gate_bias = torch.zeros((1, hidden_units), device=device)
    output_gate_bias = torch.zeros((1, hidden_units), device=device)
    c_tilda_gate_bias = torch.zeros((1, hidden_units), device=device)

    # 输出层参数
    hidden_output_weights = normal((hidden_units, output_units))
    output_bias = torch.zeros((1, output_units), device=device)

    # 将所有参数添加到字典
    parameters = {
        'fgw': forget_gate_weights,
        'igw': input_gate_weights,
        'ogw': output_gate_weights,
        'cgw': c_tilda_gate_weights,
        'fgb': forget_gate_bias,
        'igb': input_gate_bias,
        'ogb': output_gate_bias,
        'cgb': c_tilda_gate_bias,
        'how': hidden_output_weights,
        'ob': output_bias
    }

    # 设置 requires_grad=True 以启用梯度计算
    # 确保所有参数在反向传播中能够计算梯度
    for param in parameters.values():
        param.requires_grad_(True)

    return parameters



下面这一块就开始 lstm 的代码实现了，阅读这部分时确保你已经对相关的公式有所了解。

在代码内部，我们首先从参数字典中读取出相关的参数，然后将传入的当前批次数据与历史的 hidden state 进行串联。接着再依次通过每个"门"，并且计算LSTM 的 cell_state 与 hidden state。

有的同学可能就会有疑问了，代码中实现的公式和理论讲解中的公式不太一样呀，以遗忘门为例：

理论讲解中的公式是： 
$F_t = \sigma(X_t W_{xf} + H_{t-1} W_{hf} + b_f)$

对于遗忘门结构应该有两个权重参数矩阵，$W_{xf}$ 与 $W_{hf}$，而代码中只有一个参数矩阵 `fgw`，并且理论公式中，我们将 $X_t$ 与 $H_{t-1}$ 与权重矩阵相乘后才进行的拼接，而代码中却直接拼接了，这是为什么呢？我们来一起分析一下这个问题

以股票预测问题为例，我们要用过去 7 天的股票数据来预测第 8 天的股票价格，那么我们的输入数据 $X_t$ 就是一个 `batch_size * 7` 的向量，其中 `batch_size` 表示当前批次的样本数量，7 表示过去 7 天的股票价格。

那么假设 $X_t$ 是一个 $batch_size$ * 7 的向量 ， $H_{t-1}$是一个 `batch_size * hidden_units` 的向量，那么 $X_t$ 与 $H_{t-1}$ 通过 `torch.cat` 或者 `np.concatenate` 拼接后得到的新向量是 $batch_size * (7 + hidden_units)$。

对于权重矩阵，相应的，我们在上一个`code cell`定义其形状为`(input_units + hidden_units, hidden_units)`，其中 `input_units` 实际上就是 7，可以看出上这里定义的权重矩阵包含两个部分，$W_{xf} 与 W_{hf}$，前者形状为 `input_units * hidden_units`，后者形状为 `hidden_units * hidden_units`

这样，原始的两个独立的矩阵乘法和加法运算$X_t W_{xf} + H_{t-1} W_{hf}$可以被单独重写成一个矩阵乘法，即 $X_t W_{xf} + H_{t-1} W_{hf} = concat\_ dataset  W$。这种重写不仅简化了表达式，还使得实现更加高效，因为可以少维护了一个权重矩阵，在计算上更简洁。

In [ ]:
# TODO 加公式

# single lstm cell
def lstm_cell(batch_dataset, prev_hidden_state, prev_cell_state, parameters):
    # get parameters
    fgw = parameters['fgw']
    igw = parameters['igw']
    ogw = parameters['ogw']
    cgw = parameters['cgw']

    fgb = parameters['fgb']
    igb = parameters['igb']
    ogb = parameters['ogb']
    cgb = parameters['cgb']
    
    # 串联 data 和 prev_hidden_state  
    # concat_dataset = torch.cat((batch_dataset, prev_hidden_state), dim=1)
    concat_dataset = np.concatenate((batch_dataset, prev_hidden_state), axis=1)

    # forget gate activations
    F = sigmoid(np.matmul(concat_dataset, fgw) + fgb)

    # input gate activations
    I = sigmoid(np.matmul(concat_dataset, igw) + igb)

    # output gate activations
    O = sigmoid(np.matmul(concat_dataset, ogw) + ogb)

    # cell_tilda gate activations
    C_tilda = np.tanh(np.matmul(concat_dataset, cgw) + cgb)

    # 更新 cell state, hidden_state
    cell_state = F * prev_cell_state + I * C_tilda
    hidden_state = np.multiply(O, np.tanh(cell_state))

    # store four gate weights to be used in back propagation
    lstm_activations = {
        'F': F,
        'I': I,
        'O': O,
        'C_tilda': C_tilda
    }
    
    return lstm_activations, hidden_state, cell_state

In [8]:
# 输出层
# 需要注意的是，只有隐状态才会传递到输出层，而记忆元不直接参与输出计算，记忆元完全属于内部信息
def output_cell(hidden_state, parameters):
    # get hidden to output parameters
    how = parameters['how']
    ob = parameters['ob']
    # calculate the output
    output = np.matmul(hidden_state, how) + ob
    # 如果输出为概率的话，可以使用softmax函数进行归一化
    # output = softmax(output)
    return output

在定义完 `lstm_cell` 以及 `output_cell` 之后，我们在其之上定义了一个 lstm ，负责输入数据并拿到输出，`lstm`函数包含三个参数，`batch_dataset` 表示一批输入数据，`initial_state` 表示初始化状态的一个函数，parameters 表示当前模型的参数。

我们先初始化模型 state，然后依次通过 `lstm_cell` 拿到每个时间步的输出，最后通过 `output_cell` 拿到最终的输出。

In [9]:
def lstm(batch_dataset, initail_state, parameters):
    hidden_state, cell_state = initail_state
    outputs = []
    _, hidden_state, cell_state = lstm_cell(batch_dataset, hidden_state, cell_state, parameters)
    outputs.append(output_cell(hidden_state, parameters))    
    return outputs, (hidden_state, cell_state)


做完上述工作，我们已经解决了前向传播的问题，还需要封装一个简单的 RNN类来做模型参数的初始化，状态的初始化以及前向传播。

`__call__`方法的作用是使实例对象可以像调用普通函数那样，以“对象名()”的形式使用。完成 lstm 的前向传播 `forward_fn`，我们只需要将其指定为上述所定义的`lstm`函数即可

In [10]:
# 定义一个RNN 类来训练LSTM
import torch.nn.functional as F

class RNNModelScratch:
    def __init__(self, vocab_size, num_hiddens, device, get_params, init_state, forward_fn):
        self.vocab_size = vocab_size
        self.num_hiddens = num_hiddens
        self.params = get_params(vocab_size, hidden_units, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        # 根据任务不同灵活对输入数据进行预先处理
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        # X = X.type(torch.float32)
        return self.forward_fn(X, state, self.params)
    
    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [11]:
model = RNNModelScratch(vocab_size, hidden_units, device, initialize_parameters, init_lstm_state, lstm)
model

**展示模型的参数**

In [12]:
model.params

{'fgw': tensor([[-0.0129,  0.0029, -0.0024,  ..., -0.0031,  0.0026, -0.0097],
         [-0.0105, -0.0057, -0.0095,  ...,  0.0121,  0.0177, -0.0036],
         [ 0.0069,  0.0037, -0.0084,  ...,  0.0045,  0.0112,  0.0222],
         ...,
         [ 0.0178, -0.0045,  0.0048,  ..., -0.0028, -0.0112, -0.0143],
         [-0.0027,  0.0034,  0.0107,  ..., -0.0064,  0.0085,  0.0130],
         [-0.0070,  0.0030, -0.0072,  ...,  0.0096, -0.0125,  0.0097]],
        requires_grad=True),
 'igw': tensor([[ 0.0003,  0.0072, -0.0107,  ..., -0.0024, -0.0070, -0.0120],
         [ 0.0151,  0.0166, -0.0284,  ..., -0.0058, -0.0040, -0.0056],
         [-0.0079, -0.0048, -0.0048,  ..., -0.0017, -0.0081, -0.0022],
         ...,
         [-0.0239,  0.0221,  0.0033,  ...,  0.0129, -0.0150, -0.0046],
         [-0.0017, -0.0003,  0.0045,  ..., -0.0064,  0.0023,  0.0043],
         [-0.0104,  0.0113,  0.0076,  ...,  0.0147,  0.0108, -0.0085]],
        requires_grad=True),
 'ogw': tensor([[ 0.0052,  0.0051,  0.0007,  .

**恭喜你🎉**

至此你已经能完成 `lstm` 模型的前向传播了，`model.py`的内容与本篇代码类似。在下一个阶段，我们将会通过一个股票预测问题，来应用我们的lstm模型，同时加入反向传播与参数更新部分。具体代码在 `main.py`中。运行下列命令就可以开始训练了

ps: 因为过程中使用了 `swanlab` 这个工具来做可视化，所以您需要先注册一个账号并在终端中贴入你的 `api_key`

swanlab: [swanlab.cn](https://swanlab.cn/)

In [14]:
!python main.py

3852.87s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


swanlab: Tracking run with swanlab version 0.3.23                                  
swanlab: Run data will be saved locally in /Users/little1d/Desktop/Playground/LSTM-From-Scratch/notebook/swanlog/run-20241108_165219-2a23d349
swanlab: 👋 Hi Harrison, welcome to swanlab!
swanlab: Syncing run LSTM to the cloud
swanlab: 🌟 Run `swanlab watch /Users/little1d/Desktop/Playground/LSTM-From-Scratch/notebook/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@Harrison/Google-Stock-Prediction
swanlab: 🚀 View run at https://swanlab.cn/@Harrison/Google-Stock-Prediction/runs/9ukh7j8s98w9sdw6y7sis
Epoch 1, Loss: 0.2294756778412395
Epoch 2, Loss: 0.011775485281961866
Epoch 3, Loss: 0.002961122291000922
Epoch 4, Loss: 0.002451772078832922
Epoch 5, Loss: 0.0021021694523773882
Epoch 6, Loss: 0.0018974299099580902
Epoch 7, Loss: 0.0017400443749566977
Epoch 8, Loss: 0.0016426904540922907
Epoch 9, Loss: 0.0015705324921226646
Epoch 10, Loss: 0.00150444138853345